## Archive Export



In [1]:
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
from matplotlib import cm
from metrics import *
from average_meter import AverageMeter
import time
from nuscenes.utils.data_classes import PointCloud
from dh_grid import DHGrid
from sphere import Sphere
from tqdm.auto import tqdm
import os
from semantic_classes import SemanticClasses

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
def CreateGrid(bw):
    n_grid = 2 * bw
    k = 0;
    points = np.empty([2, n_grid, n_grid])
    for i in range(n_grid):
        for j in range(n_grid):
            points[0, i, j] = (np.pi*(2*i+1))/(4*bw)
            points[1, i, j] = (2*np.pi*j)/(2*bw);
            k = k + 1;
    return points

def createGrid_old(bw):
        n_grid = 2 * bw
        k = 0;
        points = np.empty([n_grid * n_grid, 2])
        for i in range(n_grid):
            for j in range(n_grid):
                points[k, 0] = (np.pi*(2*i+1))/(4*bw)
                points[k, 1] = (2*np.pi*j)/(2*bw);
                k = k + 1;
        return points
    
def convertGridToEuclidean_old(grid):
    cart_grid = np.zeros([ grid.shape[0], 3])
    cart_grid[:,0] = np.multiply(np.sin(grid[:,0]), np.cos(grid[:,1]))
    cart_grid[:,1] = np.multiply(np.sin(grid[:,0]), np.sin(grid[:,1]))
    cart_grid[:,2] = np.cos(grid[:,0])    
    return cart_grid

def mapIntensityToRGB(i):
    return cm.jet(plt.Normalize(min(i), max(i))(i))

class SamplingPointCloud(PointCloud):

    @staticmethod
    def nbr_dims() -> int:
        return 4
    
    @classmethod
    def from_file(cls, file_name: str) -> 'SamplingPointCloud':
        return None
    
    @classmethod
    def from_bw(cls, bw, scale = 100) -> 'SamplingPointCloud':
        grid = createGrid_old(bw)
        xyz_grid = convertGridToEuclidean_old(grid) * scale
        intensities = np.zeros((xyz_grid.shape[0],1))
        sampling_grid = np.hstack((xyz_grid, np.ones((xyz_grid.shape[0], 1), dtype=xyz_grid.dtype)))
        return cls(sampling_grid.T)

def create_spherical_pc(feature, trans = 0, bw = 100):
    pc = SamplingPointCloud.from_bw(bw, 1)   
    points_xyz = pc.points.T[:,0:3]
    points_xyz[:,0] = points_xyz[:,0] + trans
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points_xyz[:, 0:3])
    colors = mapIntensityToRGB(feature[:, 0])
    pcd.colors = o3d.utility.Vector3dVector(colors[:,0:3])
    return pcd    

def create_cloud_pc(cloud, trans = 0):
    cloud[:,0] = cloud[:,0] + trans
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(cloud[:, 0:3])
    if cloud.shape[1] == 4:
        colors = mapIntensityToRGB(cloud[:, 3])
        pcd.colors = o3d.utility.Vector3dVector(colors[:,0:3])
    return pcd

def compare_estimation_sphere(decoded, gt, bw = 100):  
    decoded_pc = create_spherical_pc(decoded, trans=0, bw=bw)
    gt_pc = create_spherical_pc(gt, trans=2.5, bw=bw)
    o3d.visualization.draw_geometries([decoded_pc, gt_pc])    
    
def compare_estimation_clouds(decoded, gt, bw = 100):  
    decoded_pc = create_cloud_pc(decoded, trans=0)
    gt_pc = create_cloud_pc(gt, trans=100)
    o3d.visualization.draw_geometries([decoded_pc, gt_pc])
    
def backproject_cloud(spherical, distance, bw = 100):    
    grid, _ = DHGrid.CreateGrid(bw)
    n_points = grid.shape[1] * grid.shape[2]
    cart_sphere = np.zeros([n_points, 4])
    k = 0
    for i in range(0, grid.shape[1]):
        for j in range(0, grid.shape[2]):
            dist = distance[i,j]
            if dist <= 0:
                continue
            cart_sphere[k,0] = dist * np.multiply(np.cos(grid[1,i,j]), np.sin(grid[0,i,j]))
            cart_sphere[k,1] = dist * np.multiply(np.sin(grid[1,i,j]), np.sin(grid[0,i,j]))
            cart_sphere[k,2] = dist * np.cos(grid[0,i,j])
            cart_sphere[k,3] = spherical[i,j]
            k = k + 1
    return cart_sphere  

def prepare_for_viz(cloud):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(cloud[:, 0:3])
    if cloud.shape[1] == 4:
        colors = mapIntensityToRGB(cloud[:, 3])
        pcd.colors = o3d.utility.Vector3dVector(colors[:,0:3])
    if cloud.shape[1] == 6:
        pcd.colors = o3d.utility.Vector3dVector(cloud[:,3:6] / 255.0)
    return pcd

def convert_sphere(sph, feature_idx, n_features, bw, trans = 0.0):
    sph = np.reshape(sph, (n_features, -1)).T
    
    pc = SamplingPointCloud.from_bw(bw, 1)
    points_xyz = pc.points.T[:,0:3]
    points_xyz[:,0] = points_xyz[:,0] + trans
    return np.column_stack((points_xyz, sph[:,feature_idx]))
    
def visualize_pointcloud(cloud):
    pcd = prepare_for_viz(cloud)
    o3d.visualization.draw_geometries([pcd])

def visualize_sphere(sph, feature_idx = 0, n_features = 3, bw = 100):
    points_xyzi = convert_sphere(sph, feature_idx, n_features, bw)
    visualize_pointcloud(points_xyzi)
      
def compare_estimation_sphere2(decoded, gt, bw = 100):  
    decoded_pc = convert_sphere(decoded, 0, 1, 100)
    gt_pc = convert_sphere(gt, 2, 3, 100, 2.5)
    
    decoded_pcd = prepare_for_viz(decoded_pc)
    gt_pcd = prepare_for_viz(gt_pc)
    o3d.visualization.draw_geometries([decoded_pcd, gt_pcd]) 
    
def compare_estimation_sphere3(decoded, gt, feature_idx = 0, n_features = 3, bw = 100):
    decoded_pc = convert_sphere(decoded, feature_idx, n_features, bw)
    gt_pc = convert_sphere(gt, feature_idx, n_features, bw, 2.5)
    
    decoded_pcd = prepare_for_viz(decoded_pc)
    gt_pcd = prepare_for_viz(gt_pc)
    o3d.visualization.draw_geometries([decoded_pcd, gt_pcd]) 

In [4]:
# dataroot = '/media/berlukas/SSD_1TB/s2ae/waymo'
# dataroot = '/media/berlukas/SSD_1TB/s2ae/kitti/'
dataroot = '/media/berlukas/Data2/datasets/nuscenes/processed/pointclouds'
export_ds = '/media/berlukas/Data2/datasets/nuscenes/processed/raw/'


samples = sorted(os.listdir(dataroot))
n_samples = len(samples)
cloud_features = None
for i in range(0, 1):
    sample = samples[i]
    
    sem_clouds_filename = f'{dataroot}/{sample}'
    print(f'Loading from sem clouds from {sem_clouds_filename}')
    if cloud_features is None:
        cloud_features = np.load(sem_clouds_filename, allow_pickle=True)
    else:
        features = np.load(sem_clouds_filename, allow_pickle=True)
        cloud_features = np.concatenate((cloud_features, features))    
    

Loading from sem clouds from /media/berlukas/Data2/datasets/nuscenes/processed/pointclouds/archive01.npy


In [5]:
cloud = cloud_features[2222]
visualize_pointcloud(cloud[:, [0, 1, 2, 4]])

In [6]:
# Raw export
n_clouds = len(cloud_features)
for i in tqdm(range(0,n_clouds)):
    filename = f'{export_ds}/scan{i}.npy'
    np.save(filename, cloud_features[i])

  0%|          | 0/1447 [00:00<?, ?it/s]

In [7]:
n_clouds = len(cloud_features)
for i in tqdm(range(5004, n_clouds)):
    cloud = cloud_features[i]
    # viz before
    # visualize_pointcloud(cloud[:, [0, 1, 2, 4]]) 
    
    # map semantic label
    cloud[:,4] = list(map(SemanticClasses.map_nuscenes_label, cloud[:,4]))
    
    # cap range
    ranges = np.linalg.norm(cloud[:,0:3], axis=1)
    cloud[ranges > 35, 4] = 0  
    
    # viz after
    # visualize_pointcloud(cloud[:, [0, 1, 2, 4]])
    # break
    
    filename = f'{export_ds}/scan{i}.npy'
    np.save(filename, cloud)

  0%|          | 0/6226 [00:00<?, ?it/s]

# Fix Semantic GT from exported archives

In [3]:
dataroot = '/media/berlukas/Data/data/datasets/s2ae/raw/waymo_05_09'
export_ds = '/media/berlukas/Data/data/datasets/s2ae/raw/waymo'

clouds = sorted(os.listdir(dataroot))
n_clouds = len(clouds)
for i in tqdm(range(0, n_clouds)):
    sample = clouds[i]
    filename = f'{dataroot}/{sample}'
    cloud = np.load(filename)
    
    # viz before
    # visualize_pointcloud(cloud[:, [0, 1, 2, 4]]) 
    
    # map semantic label
    cloud[:,4] = list(map(SemanticClasses.map_waymo_to_nuscenes_label, cloud[:,4]))
    
    # cap range
    ranges = np.linalg.norm(cloud[:,0:3], axis=1)
    cloud[ranges > 35, 4] = 0  
    
    # viz after
    # visualize_pointcloud(cloud[:, [0, 1, 2, 4]])
    
    filename = f'{export_ds}/{sample}'
    np.save(filename, cloud)
    

  0%|          | 0/3684 [00:00<?, ?it/s]